In [7]:

!pip install git+https://github.com/FacePerceiver/facer.git@main
!pip install timm
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import Image, display

import sys
import torch

import torch.nn.functional as F
sys.path.append('..')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install -y git
!git config --global user.name "ChinmayG123"
!git config --global user.email "chinmaygowdru2@gmail.com"


In [ ]:
!git clone https://github.com/FacePerceiver/facer.git

In [6]:
from google.colab import files
files.download('/content/drive/MyDrive/face_parsing_Chinmay.ipynb')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import facer
!pip show facer
from PIL import Image as PILImage

import os
import cv2
import numpy as np

import matplotlib.pyplot as plt


image = facer.hwc2bchw(facer.read_hwc('drive/MyDrive/downsampledimages/12.jpg')
                       ).to(device=device)  # image: 1 x 3 x h x w

face_detector = facer.face_detector('retinaface/mobilenet', device=device)
with torch.inference_mode():
    faces = face_detector(image)

# Check if faces is empty
if len(faces) == 0:
    print("No faces detected.")
else:
    facer.show_bchw(facer.draw_bchw(image, faces))

#facer.show_bchw(facer.draw_bchw(image, faces))


No faces detected.


In [ ]:
face_parser = facer.face_parser('farl/celebm/448', device=device) # optional "farl/lapa/448"


In [ ]:
with torch.inference_mode():
    faces = face_parser(image, faces)

In [ ]:
import pandas as pd


seg_logits = faces['seg']['logits']

seg_probs = seg_logits.softmax(dim=1)  # nfaces x nclasses x h x w

# Get the segmentation labels (class index) for each pixel
segmentation_labels = seg_probs.argmax(dim=1)

# Optionally, if you need to keep the same dimensions as the `seg_probs` tensor:
segmentation_labels = segmentation_labels.unsqueeze(1)

# Now you have the segmentation labels for each pixel, and you can use them for further processing or visualization.
# print("Segmentation Labels", segmentation_labels)

# Assuming `segmentation_labels` is the tensor obtained from the previous code
segmentation_labels_np = segmentation_labels.squeeze().cpu().numpy()

# Save the segmentation_labels_np as an npy file
output_file = '/content/drive/MyDrive/segmentation_masks_npy/59.npy'
np.save(output_file, segmentation_labels_np)




# Select the first channel and convert it to a NumPy array
segmentation_labels_np = segmentation_labels[0].squeeze().cpu().numpy()

# Compute the number of classes directly from the tensor
n_classes = segmentation_labels.max().item() + 1

# Choose a suitable colormap for segmentation tasks (e.g., 'viridis' or 'jet')
cmap = plt.get_cmap('viridis', n_classes)  # You can adjust the colormap as desired

# Create a new figure without displaying the plot
plt.figure(figsize=(10, 10))  # You can adjust the figure size as needed

# Plot the segmentation labels as an image using the custom colormap
plt.imshow(segmentation_labels_np, cmap=cmap, vmin=0, vmax=n_classes - 1)
plt.axis('off')  # Turn off axis labels and ticks
plt.colorbar().remove()  # Remove the colorbar

# Save the image directly using the imsave function
output_path = '/content/drive/MyDrive/segmentation_masks/59.png'
plt.imsave(output_path, segmentation_labels_np, cmap=cmap)

# Close the figure to free up resources (optional)
plt.close()











# # Set the target size to 2592 x 1728
# target_h, target_w = 1728, 2592
# # Upsample the segmentation_labels tensor to match the target size

# upsampled_segmentation_labels = F.interpolate(segmentation_labels.float(), size=(target_h, target_w), mode='nearest').long()

# # Select the first channel and convert it to a NumPy array
# upsampled_segmentation_labels_np = upsampled_segmentation_labels[0].squeeze().cpu().numpy()

# # Compute the number of classes directly from the tensor
# n_classes = upsampled_segmentation_labels.max().item() + 1

# # Choose a suitable colormap for segmentation tasks (e.g., 'viridis' or 'jet')
# cmap = plt.get_cmap('viridis', n_classes)  # You can adjust the colormap as desired

# # Create a new figure without displaying the plot
# plt.figure(figsize=(10, 10))  # You can adjust the figure size as needed

# # Plot the segmentation labels as an image using the custom colormap
# plt.imshow(upsampled_segmentation_labels_np, cmap=cmap, vmin=0, vmax=n_classes - 1)
# plt.axis('off')  # Turn off axis labels and ticks
# plt.colorbar().remove()  # Remove the colorbar

# # Save the image directly using the imsave function
# output_path = '/content/drive/MyDrive/segmentation_masks_upsampled/3.png'
# plt.imsave(output_path, upsampled_segmentation_labels_np, cmap=cmap)

# # Close the figure to free up resources (optional)
# plt.close()

# # Reset the print options to the default
# torch.set_printoptions(profile='default')







n_classes = seg_probs.size(1)
vis_seg_probs = seg_probs.argmax(dim=1).float()/n_classes*255
vis_img = vis_seg_probs.sum(0, keepdim=True)
facer.show_bhw(vis_img)
facer.show_bchw(facer.draw_bchw(image, faces))

In [ ]:
!pip install git+https://github.com/FacePerceiver/facer.git@main
!pip install timm
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import Image, display

import sys
import torch

import torch.nn.functional as F
sys.path.append('..')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:

# run on a directory of images
import facer
!pip show facer
from PIL import Image as PILImage

import os
import cv2
import numpy as np

import matplotlib.pyplot as plt



# Define the path to the folder containing the images
image_folder_path = '/content/drive/MyDrive/512/11/'

# Load all image filenames from the folder
image_filenames = os.listdir(image_folder_path)

# Face Detector
face_detector = facer.face_detector('retinaface/mobilenet', device=device)

# Face Parser
face_parser = facer.face_parser('farl/celebm/448', device=device) # optional "farl/lapa/448"



# Loop through each image and perform face detection and parsing
for filename in image_filenames:
    image_path = os.path.join(image_folder_path, filename)
    image = facer.hwc2bchw(facer.read_hwc(image_path)).to(device=device)  # image: 1 x 3 x h x w

    with torch.inference_mode():
        faces = face_detector(image)

    if len(faces) == 0:
        print(f"No faces detected in {filename}.")
    else:
        with torch.inference_mode():
            faces = face_parser(image, faces)

        seg_logits = faces['seg']['logits']
        seg_probs = seg_logits.softmax(dim=1)  # nfaces x nclasses x h x w

        # Get the segmentation labels (class index) for each pixel
        segmentation_labels = seg_probs.argmax(dim=1)

        # Optionally, if you need to keep the same dimensions as the `seg_probs` tensor:
        segmentation_labels = segmentation_labels.unsqueeze(1)

        # Now you have the segmentation labels for each pixel, and you can use them for further processing or visualization.
        # print("Segmentation Labels", segmentation_labels)

        # Assuming `segmentation_labels` is the tensor obtained from the previous code
        segmentation_labels_np = segmentation_labels.squeeze().cpu().numpy()

        # Save the segmentation_labels_np as an npy file
        output_file = os.path.join('/content/drive/MyDrive/512segmentation/11/', f'{filename}.npy')
        np.save(output_file, segmentation_labels_np)








# image = facer.hwc2bchw(facer.read_hwc('drive/MyDrive/512/1/11.jpg')
#                        ).to(device=device)  # image: 1 x 3 x h x w

# face_detector = facer.face_detector('retinaface/mobilenet', device=device)
# with torch.inference_mode():
#     faces = face_detector(image)

# # Check if faces is empty
# if len(faces) == 0:
#     print("No faces detected.")
# else:
#     facer.show_bchw(facer.draw_bchw(image, faces))

# #facer.show_bchw(facer.draw_bchw(image, faces))

# face_parser = facer.face_parser('farl/celebm/448', device=device) # optional "farl/lapa/448"

# with torch.inference_mode():
#     faces = face_parser(image, faces)

# import pandas as pd


# seg_logits = faces['seg']['logits']

# seg_probs = seg_logits.softmax(dim=1)  # nfaces x nclasses x h x w

# # Get the segmentation labels (class index) for each pixel
# segmentation_labels = seg_probs.argmax(dim=1)

# # Optionally, if you need to keep the same dimensions as the `seg_probs` tensor:
# segmentation_labels = segmentation_labels.unsqueeze(1)

# # Now you have the segmentation labels for each pixel, and you can use them for further processing or visualization.
# # print("Segmentation Labels", segmentation_labels)

# # Assuming `segmentation_labels` is the tensor obtained from the previous code
# segmentation_labels_np = segmentation_labels.squeeze().cpu().numpy()

# # Save the segmentation_labels_np as an npy file
# output_file = '/content/drive/MyDrive/512/1/11.jpg.npy'
# np.save(output_file, segmentation_labels_np)




# # Select the first channel and convert it to a NumPy array
# segmentation_labels_np = segmentation_labels[0].squeeze().cpu().numpy()

# # Compute the number of classes directly from the tensor
# n_classes = segmentation_labels.max().item() + 1

# # Choose a suitable colormap for segmentation tasks (e.g., 'viridis' or 'jet')
# cmap = plt.get_cmap('viridis', n_classes)  # You can adjust the colormap as desired

# # Create a new figure without displaying the plot
# plt.figure(figsize=(10, 10))  # You can adjust the figure size as needed

# # Plot the segmentation labels as an image using the custom colormap
# plt.imshow(segmentation_labels_np, cmap=cmap, vmin=0, vmax=n_classes - 1)
# plt.axis('off')  # Turn off axis labels and ticks
# plt.colorbar().remove()  # Remove the colorbar

# # Save the image directly using the imsave function
# output_path = '/content/drive/MyDrive/segmentation_masks/59.png'
# plt.imsave(output_path, segmentation_labels_np, cmap=cmap)

# # Close the figure to free up resources (optional)
# plt.close()











# # Set the target size to 2592 x 1728
# target_h, target_w = 1728, 2592
# # Upsample the segmentation_labels tensor to match the target size

# upsampled_segmentation_labels = F.interpolate(segmentation_labels.float(), size=(target_h, target_w), mode='nearest').long()

# # Select the first channel and convert it to a NumPy array
# upsampled_segmentation_labels_np = upsampled_segmentation_labels[0].squeeze().cpu().numpy()

# # Compute the number of classes directly from the tensor
# n_classes = upsampled_segmentation_labels.max().item() + 1

# # Choose a suitable colormap for segmentation tasks (e.g., 'viridis' or 'jet')
# cmap = plt.get_cmap('viridis', n_classes)  # You can adjust the colormap as desired

# # Create a new figure without displaying the plot
# plt.figure(figsize=(10, 10))  # You can adjust the figure size as needed

# # Plot the segmentation labels as an image using the custom colormap
# plt.imshow(upsampled_segmentation_labels_np, cmap=cmap, vmin=0, vmax=n_classes - 1)
# plt.axis('off')  # Turn off axis labels and ticks
# plt.colorbar().remove()  # Remove the colorbar

# # Save the image directly using the imsave function
# output_path = '/content/drive/MyDrive/segmentation_masks_upsampled/3.png'
# plt.imsave(output_path, upsampled_segmentation_labels_np, cmap=cmap)

# # Close the figure to free up resources (optional)
# plt.close()

# # Reset the print options to the default
# torch.set_printoptions(profile='default')







n_classes = seg_probs.size(1)
vis_seg_probs = seg_probs.argmax(dim=1).float()/n_classes*255
vis_img = vis_seg_probs.sum(0, keepdim=True)
facer.show_bhw(vis_img)
facer.show_bchw(facer.draw_bchw(image, faces))




No faces detected in 1.jpg.
No faces detected in 12.jpg.
No faces detected in 0.jpg.
No faces detected in 2.jpg.
